In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import scipy as sp
import datetime as dt

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report, confusion_matrix, accuracy_score
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, precision_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import statsmodels.api as sm

import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy.optimize import fsolve
from scipy.optimize import curve_fit

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
from statsmodels.stats.stattools import jarque_bera

import seaborn as sns
import yfinance as yf

#### Step 1: Data Importing

In [2]:
eursek_tick_data = pd.read_csv("EURSEK_tick_UTC+0_00_2019-Parse.csv.zst")

In [3]:
eursek_tick_data

,UTC,AskPrice,BidPrice,AskVolume,BidVolume
0,2019-01-01T22:03:06.044+00:00,10.15908,10.17841,0.75,0.75
1,2019-01-01T22:03:14.888+00:00,10.15903,10.17834,0.75,0.75
2,2019-01-01T22:03:34.932+00:00,10.15904,10.17835,0.75,0.75
3,2019-01-01T22:03:54.888+00:00,10.15905,10.17836,0.75,0.75
4,2019-01-01T22:04:02.688+00:00,10.15826,10.17825,0.75,0.75
...,...,...,...,...,...
23626115,2019-12-31T21:58:49.588+00:00,10.48746,10.51634,1.30,1.00
23626116,2019-12-31T21:58:52.767+00:00,10.48746,10.51594,1.30,1.00
23626117,2019-12-31T21:58:58.312+00:00,10.48746,10.51555,1.30,1.00
23626118,2019-12-31T21:59:00.557+00:00,10.48746,10.51514,1.30,1.00


In [4]:
eursek_tick_data['AskVolume'] = eursek_tick_data['AskVolume'].groupby(eursek_tick_data.index // 10).cumsum()
eursek_tick_data['BidVolume'] = eursek_tick_data['BidVolume'].groupby(eursek_tick_data.index // 10).cumsum()

In [5]:
len(eursek_tick_data)
data = eursek_tick_data.loc[::10]
data.index = pd.to_datetime(data['UTC'])

In [6]:
data.drop('UTC',axis=1)
data['MidPrice'] = (data['AskPrice'] + data['BidPrice'])/2

C:\Users\camer\AppData\Local\Temp\ipykernel_33332\3205618487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MidPrice'] = (data['AskPrice'] + data['BidPrice'])/2


In [7]:
final_bars = pd.DataFrame(index=data.index)
final_bars['log close'] = np.log(data['MidPrice'])
final_bars['log std'] = final_bars['log close'].rolling(50).std()
final_bars['close'] = data['MidPrice']
final_bars = final_bars.dropna()
final_bars.index = final_bars.index.tz_localize(None)

In [8]:
final_bars

,log close,log std,close
UTC,,,
2019-01-01 23:05:17.013,2.319520,0.000154,10.170790
2019-01-01 23:05:26.037,2.319422,0.000155,10.169790
2019-01-01 23:05:27.004,2.319451,0.000154,10.170085
2019-01-01 23:05:29.486,2.319476,0.000152,10.170340
2019-01-01 23:05:36.650,2.319458,0.000148,10.170160
...,...,...,...
2019-12-31 21:53:24.510,2.351642,0.000275,10.502800
2019-12-31 21:54:26.787,2.351404,0.000274,10.500300
2019-12-31 21:55:34.797,2.351594,0.000274,10.502300


#### Step 2: Filter data, form vertical barrier

In [9]:
def sym_cusum(raw_data, h):
    filtered_events_indices = []
    s_pos = 0
    s_neg = 0
    diff = raw_data.diff()
    count = 0
    for i in diff.index[1:]:
        count+=1
        s_pos = max(0, s_pos + diff.loc[i])
        s_neg = min(0, s_neg + diff.loc[i])
        if s_neg < -h:
            s_neg = 0
            filtered_events_indices.append(i)
        elif s_pos > h:
            s_pos = 0
            filtered_events_indices.append(i)
        if count%100000 == 0:
            print(count)
    return pd.DatetimeIndex(filtered_events_indices)

def get_vertical_barrier(close, t_events, num_days):
    t1 = close.index.searchsorted(t_events+pd.Timedelta(days=num_days))
    t1 = t1[t1<close.shape[0]]
    return pd.Series(close.index[t1], index=t_events[:t1.shape[0]])

In [10]:
# get relevant events
event_filter_diff_min_h = final_bars['log std'].mean()/2
t_events = sym_cusum(final_bars['log close'], event_filter_diff_min_h)

t_events
# get vertical barriers
t1 = get_vertical_barrier(final_bars['log close'], t_events, 1)


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000


#### Step 3: RSI/MACD for side labels

In [11]:
# side of trade will be decided from a mixture of indicators - MACD, RSI, Crossing SMAs, Bollinger bands, Stochastic oscillator

# MACD
fast_window_macd = 12
slow_window_macd = 26
macd_signal_window = 9

raw_data = pd.DataFrame({'log close': final_bars['log close']})

raw_data['fast_ema'] = raw_data['log close'].ewm(span=fast_window_macd).mean()
raw_data['slow_ema'] = raw_data['log close'].ewm(span=slow_window_macd).mean()
raw_data['macd'] = raw_data['fast_ema'] - raw_data['slow_ema']
raw_data['macd_ema'] = raw_data['macd'].ewm(span=macd_signal_window).mean()

long_signals_macd = raw_data['macd'] >= raw_data['macd_ema'] 
short_signals_macd = raw_data['macd'] < raw_data['macd_ema'] 
raw_data.loc[long_signals_macd, 'side_macd'] = 1
raw_data.loc[short_signals_macd, 'side_macd'] = -1

# RSI
rsi_window = 14

raw_data['diff'] = raw_data['log close'].diff()
raw_data['gain'] = raw_data['diff'].apply(lambda x: x if x >= 0 else 0)
raw_data['loss'] = raw_data['diff'].apply(lambda x: -x if x < 0 else 0)
raw_data['avg_gain'] = raw_data['gain'].rolling(rsi_window).mean()
raw_data['avg_loss'] = raw_data['loss'].rolling(rsi_window).mean()
raw_data['rsi'] = 100 - 100/(1+raw_data['avg_gain']/raw_data['avg_loss'])

long_signals_rsi = raw_data['rsi'] <= 30
short_signals_rsi = raw_data['rsi'] >= 70
raw_data.loc[long_signals_macd, 'side_rsi'] = 1
raw_data.loc[short_signals_macd, 'side_rsi'] = -1
raw_data['side_rsi'] = raw_data['side_rsi'].fillna(method='ffill')

# SMAs
fast_window_sma = 20
slow_window_sma = 50
raw_data['fast_mavg_sma'] = raw_data['log close'].rolling(window=fast_window_sma, min_periods=fast_window_sma, center=False).mean()
raw_data['slow_mavg_sma'] = raw_data['log close'].rolling(window=slow_window_sma, min_periods=slow_window_sma, center=False).mean()

long_signals_sma = raw_data['fast_mavg_sma'] >= raw_data['slow_mavg_sma'] 
short_signals_sma = raw_data['fast_mavg_sma'] < raw_data['slow_mavg_sma'] 
raw_data.loc[long_signals_sma, 'side_sma'] = 1
raw_data.loc[short_signals_sma, 'side_sma'] = -1

# Bollinger bands
bb_window = 20
std_no = 2

raw_data['mavg_bb'] = raw_data['log close'].rolling(window=bb_window, min_periods=bb_window, center=False).mean()
raw_data['mstd_bb'] = raw_data['log close'].rolling(window=bb_window, min_periods=bb_window, center=False).std()
raw_data['BOLU'] = raw_data['mavg_bb'] + std_no * raw_data['mstd_bb']
raw_data['BOLD'] = raw_data['mavg_bb'] - std_no * raw_data['mstd_bb']

long_signals_bb = raw_data['log close'] <= raw_data['BOLD'] 
short_signals_bb = raw_data['log close'] >= raw_data['BOLU'] 
raw_data.loc[long_signals_bb, 'side_bb'] = 1
raw_data.loc[short_signals_bb, 'side_bb'] = -1
raw_data['side_bb'] = raw_data['side_bb'].fillna(method='ffill')

# Stochastic oscillator
so_window = 14
so_smoothing_window = 3

raw_data['fast_so'] = 100*(raw_data['log close'] - raw_data['log close'].rolling(so_window).min())/(raw_data['log close'].rolling(so_window).max() - raw_data['log close'].rolling(so_window).min())
raw_data['slow_so'] = raw_data['fast_so'].rolling(so_smoothing_window).mean()

long_signals_so = (raw_data['fast_so'] >= raw_data['slow_so']) & (raw_data['fast_so'] <= 20)
short_signals_so = (raw_data['fast_so'] < raw_data['slow_so']) & (raw_data['fast_so'] >= 80)
raw_data.loc[long_signals_so, 'side_so'] = 1
raw_data.loc[short_signals_so, 'side_so'] = -1
raw_data['side_so'] = raw_data['side_so'].fillna(method='ffill')

raw_data = raw_data.dropna()
sides = raw_data[['side_macd','side_rsi','side_sma','side_bb','side_so']]
sides['side_majority'] = sides.mode(axis=1)
sides['side_confidence'] = sides.mean(axis=1).abs()
sides['side_confidence'].value_counts()

C:\Users\camer\AppData\Local\Temp\ipykernel_33332\1313541155.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sides['side_majority'] = sides.mode(axis=1)
C:\Users\camer\AppData\Local\Temp\ipykernel_33332\1313541155.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sides['side_confidence'] = sides.mean(axis=1).abs()


side_confidence
0.333333    1477479
0.666667     849015
1.000000      35320
Name: count, dtype: int64

In [12]:
# choose what side we are going to use
side = sides['side_majority']
side

UTC
2019-01-01 23:11:39.163    1.0
2019-01-01 23:11:40.277    1.0
2019-01-01 23:11:42.445    1.0
2019-01-01 23:11:43.221   -1.0
2019-01-01 23:11:44.388   -1.0
                          ... 
2019-12-31 21:53:24.510    1.0
2019-12-31 21:54:26.787    1.0
2019-12-31 21:55:34.797    1.0
2019-12-31 21:56:22.928    1.0
2019-12-31 21:58:27.413    1.0
Name: side_majority, Length: 2361814, dtype: float64

#### Step 4: Triple barrier

In [13]:
def apply_ptsl_on_t1(close, events, ptSl, molecule): # (ptsl means profit taking and stop loss)
    '''
    close: close price data - horizontal barriers will be calculated directly from close return vol
    events: vertical barrier TIMES 
    ptSl: list of 1 or 0 indicating if upper or lower barriers should be activated
    '''
    events = events.loc[molecule] # to do with multiprocessing
    out = events[['t1']].copy(deep=True)
    if ptSl[0] > 0:
        pt = ptSl[0] * events['trgt']
    else:
        pt = np.infty * events['trgt']
    if ptSl[1] > 0:
        sl = -ptSl[1] * events['trgt']
    else:
        sl = - np.infty * events['trgt']
    for loc, t1 in events['t1'].fillna(close.index[-1]).items():
        df0 = close[loc:t1] # takes all the close prices between the time in events['t1'] (the index) and the corresponding barrier - remember the indexed times here are equivalent to the cusum filtered times, and the end barriers are approx a day later
        df0 = (df0/close[loc] - 1) * events.at[loc,'side'] # checks all the returns at once - comparing to the starting price at the beginning of this triple barrier scan
        out.loc[loc,'sl'] = df0[df0<sl[loc]].index.min()
        out.loc[loc,'pt'] = df0[df0>pt[loc]].index.min() # not sl and pt are constant across a barrier search ('box') - taken as the sl/pt at the time of box creation
    return out

def get_events(close, t_events, ptSl, trgt, minRet, numThreads, t1 = False, side = None):
    # If side is None, it means we are trying to learn the side (position) we should take, if we already know the side and are getting the events for the labelling of a secondary model, we will have an argument for side
    
    trgt = trgt.loc[t_events] # ensures we only care about the trgt values for which our cusum filter has identified
    trgt = trgt[trgt>minRet] # not particularly important, just some lower limit of return required

    if t1 is False:
        t1 = pd.Series(pd.NaT,index=t_events)
    if side is None:
        side = pd.Series(1,index=trgt.index)
    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side}, axis=1).dropna(subset=['trgt'])
    df0 = apply_ptsl_on_t1(close, events, ptSl, events.index)
  #  df0 = mpPandsObj(func=applyPtSlOnT1, pdObj=('molecule', events.index), numThreads=numThreads, close=close, events=events, ptSl=ptSl) # this function will be implemented in final chapter
    events['t1'] = df0.dropna(how='all').min(axis=1) # finds the first barrier touch out of either it being a vertical, sl, or pt
    if side is None:
        events = events.drop('side', axis=1)
    return events

In [14]:
final_bars

,log close,log std,close
UTC,,,
2019-01-01 23:05:17.013,2.319520,0.000154,10.170790
2019-01-01 23:05:26.037,2.319422,0.000155,10.169790
2019-01-01 23:05:27.004,2.319451,0.000154,10.170085
2019-01-01 23:05:29.486,2.319476,0.000152,10.170340
2019-01-01 23:05:36.650,2.319458,0.000148,10.170160
...,...,...,...
2019-12-31 21:53:24.510,2.351642,0.000275,10.502800
2019-12-31 21:54:26.787,2.351404,0.000274,10.500300
2019-12-31 21:55:34.797,2.351594,0.000274,10.502300


In [15]:
# get triple barriers
trgt = final_bars['log std']
events = get_events(final_bars['log close'], t_events, [2,2], trgt, 0.0001, 1, t1, side)
events

,t1,trgt,side
2019-01-01 23:05:26.037,2019-01-02 23:05:29.307,0.000155,NaN
2019-01-01 23:05:29.486,2019-01-02 23:05:30.501,0.000152,NaN
2019-01-01 23:05:46.040,2019-01-02 23:06:44.209,0.000146,NaN
2019-01-01 23:05:49.812,2019-01-02 23:06:44.209,0.000146,NaN
2019-01-01 23:07:17.985,2019-01-02 23:07:19.294,0.000148,NaN
...,...,...,...
2019-12-31 21:52:30.784,NaT,0.000274,1.0
2019-12-31 21:53:24.510,NaT,0.000275,1.0
2019-12-31 21:54:26.787,NaT,0.000274,1.0
2019-12-31 21:55:34.797,NaT,0.000274,1.0


In [16]:
events = events.dropna()
events

,t1,trgt,side
2019-01-01 23:11:39.163,2019-01-02 06:07:09.062,0.000269,1.0
2019-01-01 23:11:40.277,2019-01-02 06:07:07.950,0.000270,1.0
2019-01-01 23:11:42.445,2019-01-02 06:05:02.559,0.000271,1.0
2019-01-01 23:11:43.221,2019-01-02 06:04:38.413,0.000274,-1.0
2019-01-01 23:11:45.408,2019-01-02 06:04:38.413,0.000281,-1.0
...,...,...,...
2019-12-31 21:28:32.819,2019-12-31 21:31:09.938,0.000147,1.0
2019-12-31 21:28:34.811,2019-12-31 21:31:09.938,0.000135,1.0
2019-12-31 21:28:38.066,2019-12-31 21:31:09.938,0.000121,1.0
2019-12-31 21:30:00.566,2019-12-31 21:30:33.796,0.000121,1.0


In [17]:
events['side'].value_counts()

side
 1.0    135882
-1.0    128278
Name: count, dtype: int64

#### Step 5: Meta labelling - remember we may want some 

In [18]:
def get_bins(events, close):
    '''
    events.index: each triple barrier events start time
    events['t1']: events end time (the barrier hit)
    events['trgt']: horizontal barriers
    events['side'] (optional): pre-found side learnt from (separate) primary model 
    '''
    events = events.dropna(subset=['t1'])
    px = events.index.union(events['t1'].values).drop_duplicates() # taking all the index times and event times into one set and dropping any duplicates (as we will only need prices at these times)
    px = close.reindex(px, method='bfill') # just take the close prices at the relevant times - as defined above - then just deal with the odd nan by backfilling
    
    out = pd.DataFrame(index=events.index) # dataframe will have all the barrier hits from the events dataframe (as returned by the getEvents function)
    out['ret'] = px.loc[events['t1'].values].values - px.loc[events.index]  # log return takes return from the initial indexed time to the barrier hit
    out['hit'] = np.where((out['ret'] > events['trgt']) | (out['ret'] < -1*events['trgt']), 1, 0)
    
    if 'side' in events:
        out['ret'] *= events['side'] # we are not double multiplying here - remember these returns have been calculated fresh from the close in this function
    out['bin'] = np.sign(out['ret']) # remember this is the return specifically for barrier hits
    
    if 'side' in events:
        out.loc[out['ret'] <= 0, 'bin'] = 0
    else:
        out['bin'] = np.where(out['hit'] == 1, out['bin'], 0)
        
    return out

In [19]:
bins = get_bins(events, final_bars['log close'])

In [20]:
bins

,ret,hit,bin
2019-01-01 23:11:39.163,0.001262,1,1.0
2019-01-01 23:11:40.277,0.001253,1,1.0
2019-01-01 23:11:42.445,0.001269,1,1.0
2019-01-01 23:11:43.221,-0.001279,1,0.0
2019-01-01 23:11:45.408,-0.001318,1,0.0
...,...,...,...
2019-12-31 21:28:32.819,0.000695,1,1.0
2019-12-31 21:28:34.811,0.000734,1,1.0
2019-12-31 21:28:38.066,0.000679,1,1.0
2019-12-31 21:30:00.566,-0.000573,1,0.0


In [21]:
print('bins', bins['bin'].value_counts())
print('hits', bins['hit'].value_counts())

bins bin
0.0    135168
1.0    128992
Name: count, dtype: int64
hits hit
1    262717
0      1443
Name: count, dtype: int64


In [22]:
bins.to_csv('target_labels.csv')

#### Step 6: Feature engineering

When examining features originating from the time series, we want a time series that has better statistical properties, i.e. is stationary. However, fully differencing the data can result in memory and therefore information loss, therefore we wish to fractionally difference the data just enough so that it is classed as stationary within the 95% confidence interval.

In [ ]:
# import currencies we will examine and will also be fractionally differenced
eurusd_tick_data = pd.read_csv("EURUSD_tick_UTC+0_00_2019-Parse.csv.zst")
usdsek_tick_data = pd.read_csv("USDSEK_tick_UTC+0_00_2019-Parse.csv.zst")

In [ ]:
eurusd_tick_data['AskVolume'] = eurusd_tick_data['AskVolume'].groupby(eurusd_tick_data.index // 10).cumsum()
usdsek_tick_data['BidVolume'] = usdsek_tick_data['BidVolume'].groupby(usdsek_tick_data.index // 10).cumsum()
usdsek_tick_data['AskVolume'] = usdsek_tick_data['AskVolume'].groupby(usdsek_tick_data.index // 10).cumsum()
eurusd_tick_data['BidVolume'] = eurusd_tick_data['BidVolume'].groupby(eurusd_tick_data.index // 10).cumsum()

In [ ]:
features = data
features = features.drop('UTC',axis=1)
features.index.tz_localize(None)
features.columns = features.columns + 'EURSEK'

eurusd_tick_data.index = pd.to_datetime(eurusd_tick_data['UTC'])
eurusd_tick_data.drop('UTC',axis=1)
eurusd_tick_data.index.tz_localize(None)
eurusd_tick_data.columns = eurusd_tick_data.columns + 'EURUSD'

usdsek_tick_data.index = pd.to_datetime(usdsek_tick_data['UTC'])
usdsek_tick_data.drop('UTC',axis=1)
usdsek_tick_data.index.tz_localize(None)
usdsek_tick_data.columns = usdsek_tick_data.columns + 'USDSEK'

In [ ]:
features = pd.merge_asof(features, eurusd_tick_data, left_index=True, right_index=True)

In [ ]:
features = pd.merge_asof(features, usdsek_tick_data, left_index=True, right_index=True)

In [ ]:
features = features.drop(['UTCEURUSD','UTCUSDSEK'], axis=1)
features

In [ ]:
features['MidPriceEURUSD'] = (features['BidPriceEURUSD'] + features['AskPriceEURUSD'])/2
features['MidPriceUSDSEK'] = (features['BidPriceUSDSEK'] + features['AskPriceUSDSEK'])/2
features[['logCloseEURSEK','logCloseUSDSEK','logCloseEURUSD']] = np.log(features[['MidPriceEURSEK','MidPriceUSDSEK','MidPriceEURUSD']])
features[['logStdEURSEK','logStdUSDSEK','logStdEURUSD']] = features[['logCloseEURSEK','logCloseUSDSEK','logCloseEURUSD']].rolling(50).std()
features = features.dropna()
features

In [ ]:
def getWeights(d, size):
    w = [1.]
    for k in range(1,size):
        w_=-w[-1]*(d-k+1)/k
        w.append(w_)
    w=np.array(w[::-1]).reshape(-1,1)
    return w

def fracDiff(series, d, thres=0.01):
    # 1 - Compute weights for longest series, ie weights up until T
    w = getWeights(d, series.shape[0])  # note that the 1, indicating the 'first' binomial weight; the one also attached to the most recent data point, is the final element in the array.
    # 2 - Weight loss threshold calculations
    w_=np.cumsum(abs(w))
    lamda=w_/w_[-1]
    skip=lamda[lamda>thres].shape[0] # the higher the threshold, the fewer points are greater than the tthreshold, therefore we 'skip' fewer points, so ou skip index (used below) starts from a lower value, as we skip fewer initial points
    #3 - apply weights to find each fractionally differentiated value, excluding the skipped values
    df={}
    for name in series.columns:
        seriesF = series[[name]].fillna(method='ffill').dropna() # going through supplied dataframe column by column
        X_tilda=pd.Series(index=seriesF.iloc[skip:seriesF.shape[0]].index)
        for iloc in range(skip,seriesF.shape[0]): # ranging from after skipped values to most recent values
            loc=seriesF.index[iloc] # time of current int index
            if not np.isfinite(series.loc[loc,name]):
                continue
            # Now we look at the current time for which we want to calculate X_tilda
            X_tilda[loc] = np.dot(w[-(iloc+1):,:].T, seriesF.loc[:loc])[0,0] # dotting the latter weights in the transposed array (so the last weight here will still be 1) with all the past times up until the current time
        df[name] = X_tilda.copy(deep=True)
    df=pd.concat(df,axis=1)
    return df

def getWeights_FFD(d,thres):
    w = [1.]
    k=1
    while w[-1] >= thres:
        w_=-w[-1]*(d-k+1)/k
        w.append(w_)
        k+=1
    w=np.array(w[::-1]).reshape(-1,1)
    return w

def fracDiff_FFD(series, d, thres=1e-5):
    # 1 - Compute weights for longest series, ie weights up until T
    print('getting weights with d=',d)
    w=getWeights_FFD(d,thres)
    print('weights obtained')
    width=len(w)-1 # this is how many memory terms we will have for X tilda at all t
    # 2 - apply weights to find each fractionally differentiated value, excluding the skipped (equal to the width) values
    df={}
    for name in series.columns: # for if there are multiple time series in our df we wish to difference
        seriesF = series[[name]].fillna(method='ffill').dropna()
        X_tilda=pd.Series(index=seriesF.iloc[width:seriesF.shape[0]].index)
        for iloc1 in range(width,seriesF.shape[0]):
            loc0, loc1 = seriesF.index[iloc1-width], seriesF.index[iloc1] # times at beginning of window and end of window
            if not np.isfinite(series.loc[loc1,name]):
                continue
            X_tilda[loc1] = np.dot(w.T, seriesF.loc[loc0:loc1])[0,0]
        df[name]=X_tilda.copy(deep=True)
    df=pd.concat(df,axis=1)
    return df

def plotMinFFD(series,column_plot,thres=1e-2):
    from statsmodels.tsa.stattools import adfuller
    out=pd.DataFrame(columns=['adfStat','pVal','lags','nObs','95% conf','corr'])
    for d in np.linspace(0,0.7,8):
        df1=np.log(series[[column_plot]]).resample('1D').last() # resamples series to daily instead of whatever it was before and uses log prices
        df1=np.log(series[[column_plot]])
        df1=series[[column_plot]]
        df2=fracDiff_FFD(df1,d,thres) # our new fractionally differentiated series using fixed window method
        corr=np.corrcoef(df1.loc[df2.index,column_plot],df2[column_plot])[0,1] # correlation between differenced series and original series - we want this to be high as this indicates the series are similar with regards to information, therefore we have not lost a lot of memory for high correlation coefficients
        df2=adfuller(df2[column_plot],maxlag=1,regression='c',autolag=None)
        out.loc[d]=list(df2[:4]) + [df2[4]['5%']] + [corr] # just takes returns from adfuller function and puts into nice list
    plt.figure()
    out[['adfStat','corr']].plot(secondary_y='adfStat',xlabel='Fraction Differentiated Amount',ylabel='adf stat')
    plt.grid(axis='both')
    plt.axhline(out['95% conf'].mean(),linewidth=1,color='r',linestyle='dotted')
    plt.title(column_plot)
    return

In [ ]:
plotMinFFD(features,'logCloseEURSEK',1e-5)

In [ ]:
features['logCloseEURSEKFracDiff'] = fracDiff_FFD(features[['logCloseEURSEK']],0)

In [ ]:
features = features.dropna()

In [ ]:
print('p-value: ', adfuller(features['logCloseEURSEKFracDiff'],maxlag=1,regression='c',autolag=None))

In [ ]:
features['logCloseUSDSEKFracDiff'] = fracDiff_FFD(features[['logCloseUSDSEK']],0.4)
features = features.dropna()
print('p-value: ', adfuller(features['logCloseUSDSEKFracDiff'],maxlag=1,regression='c',autolag=None))

In [ ]:
features['logCloseEURUSDFracDiff'] = fracDiff_FFD(features[['logCloseEURUSD']],0.2)
features = features.dropna()
print('p-value: ', adfuller(features['logCloseEURUSDFracDiff'],maxlag=1,regression='c',autolag=None))

In [ ]:
features = series_fracdiff
features

#### Step 6.1 - TS features (self)

In [ ]:
# Volatility
features['volatility_50_EURSEK_fracDiff'] = features['logCloseEURSEKFracDiff'].rolling(window=50).std()
features['volatility_25_EURSEK_fracDiff'] = features['logCloseEURSEKFracDiff'].rolling(window=25).std()
features['volatility_10_EURSEK_fracDiff'] = features['logCloseEURSEKFracDiff'].rolling(window=10).std()

# Serial Correlation (Takes about 4 minutes)
window_autocorr = 50

features['autocorr_1_EURSEK_fracDiff'] = features['logCloseEURSEKFracDiff'].rolling(window=window_autocorr).apply(lambda x: x.autocorr(lag=1), raw=False)
print('done lag 1')
features['autocorr_2_EURSEK_fracDiff'] = features['logCloseEURSEKFracDiff'].rolling(window=window_autocorr).apply(lambda x: x.autocorr(lag=2), raw=False)
print('done lag 2')
features['autocorr_3_EURSEK_fracDiff'] = features['logCloseEURSEKFracDiff'].rolling(window=window_autocorr).apply(lambda x: x.autocorr(lag=3), raw=False)
print('done lag 3')
features['autocorr_4_EURSEK_fracDiff'] = features['logCloseEURSEKFracDiff'].rolling(window=window_autocorr).apply(lambda x: x.autocorr(lag=4), raw=False)
print('done lag 4')
features['autocorr_5_EURSEK_fracDiff'] = features['logCloseEURSEKFracDiff'].rolling(window=window_autocorr).apply(lambda x: x.autocorr(lag=5), raw=False)
print('done lag 5')

#### Step 6.2: Microstructural indicators (self and via USD currencies)

In [ ]:
# Spread, volume imbalance etc

# Spread
features['spread_EURSEK'] = features['AskPriceEURSEK'] - features['BidPriceEURSEK']
features['spread_USDSEK'] = features['AskPriceUSDSEK'] - features['BidPriceUSDSEK']
features['spread_EURUSD'] = features['AskPriceEURUSD'] - features['BidPriceEURUSD']

# Volume imbalance
features['volume_imbalance_EURSEK'] = features['BidVolumeEURSEK'] - features['AskVolumeEURSEK']
features['volume_imbalance_USDSEK'] = features['BidVolumeUSDSEK'] - features['AskVolumeUSDSEK']
features['volume_imbalance_EURUSD'] = features['BidVolumeEURUSD'] - features['AskVolumeEURUSD']
features['volume_sum_EURSEK'] = features['BidVolumeEURSEK'] + features['AskVolumeEURSEK']
features['volume_sum_USDSEK'] = features['BidVolumeUSDSEK'] + features['AskVolumeUSDSEK']
features['volume_sum_EURUSD'] = features['BidVolumeEURUSD'] + features['AskVolumeEURUSD']
features['volume_imbalance_EURSEK_normalized'] = features['volume_imbalance_EURSEK']/features['volume_sum_EURSEK']
features['volume_imbalance_USDSEK_normalized'] = features['volume_imbalance_USDSEK']/features['volume_sum_USDSEK']
features['volume_imbalance_EURUSD_normalized'] = features['volume_imbalance_EURUSD']/features['volume_sum_EURUSD']
features['EURUSD_to_USDSEK_flow'] = features['volume_imbalance_EURUSD_normalized'] + features['volume_imbalance_USDSEK_normalized']

# Picing mismatch
features['crossPrice'] = features['MidPriceEURUSD'] * features['MidPriceUSDSEK']
features['priceMisMatch'] = features['crossPrice'] - features['MidPriceEURSEK']

In [ ]:
features

#### Step 6.3: Entropy indicators (self and via USD currencies)

In [ ]:
# from AFML
def quartile_encode(df, column):
    quartile_labels = ['1', '2', '3', '4']

    # Use qcut to create a new column with quartile labels
    df[column + '_quartile'] = pd.qcut(df[column], q=4, labels=quartile_labels)
    
    return df

def match_length(msg, i, n):
    subS=''
    for l in range(n):
        msg1 = msg[i:i+l+1]
        for j in range(i-n, i):
            msg0 = msg[j:j+l+1]
            if msg1==msg0:
                subS = msg1
                break
    return len(subS)+1, subS

def konto(msg, window=None):
    out = {'num':0, 'sum':0, 'subS':[]}
    if not isinstance(msg,str):
        msg=''.join(map(str,msg))
    if window is None:
        points = range(1,len(msg)//2+1)
    else:
        window = min(window, len(msg)/2)
        points = range(window, len(msg)-window+1)
    for i in points:
        if window is None:
            l, msg_ = match_length(msg, i, i)
            out['sum']+=np.log2(i+1)/l
        else:
            l, msg_ = match_length(msg, i, window)
            out['sum']+=np.log2(window+1)/l
        out['subS'].append(msg_)
        out['num']+=1
        if i%100000 == 0:
            print(i)
    out['h'] = out['sum']/out['num']
    out['r'] = 1-out['h']/np.log2(len(msg))
    return out

msg = '1234432342123341233'
konto(msg)

Will just use scipy entropy for speed

In [ ]:
features['entropyEURSEK'] = features['logCloseEURSEKFracDiff'].rolling(window=100).apply(lambda x: sp.stats.entropy(np.histogram(x, bins='auto')[0]))
features['entropyUSDSEK'] = features['logCloseUSDSEKFracDiff'].rolling(window=100).apply(lambda x: sp.stats.entropy(np.histogram(x, bins='auto')[0]))
features['entropyEURUSD'] = features['logCloseEURUSDFracDiff'].rolling(window=100).apply(lambda x: sp.stats.entropy(np.histogram(x, bins='auto')[0]))

In [ ]:
features.index = features.index.tz_localize(None)

In [ ]:
sides.tz_localize(None)

In [ ]:
features = features.join(sides, how='inner')

In [ ]:
features = features.shift(1) # remove look ahead bias
features = features.dropna()

#### Step7: Model with sample weighting
Try both random fotest and bagged SVM

In [ ]:
model_data = bins.join(features, how='inner')
model_data = model_data.dropna()

In [ ]:
model_data.to_csv("eursek_signal_data.csv")

In [ ]:
y = model_data['bin']
y.value_counts()

In [ ]:
features.columns

In [ ]:
X = model_data[['spread_EURSEK', 'spread_USDSEK', 'spread_EURUSD', 
                'volume_imbalance_EURSEK_normalized', 'EURUSD_to_USDSEK_flow', 
                'priceMisMatch', 'entropyEURSEK', 'entropyUSDSEK',
                'volatility_50_EURSEK_fracDiff', 'volatility_25_EURSEK_fracDiff', 'volatility_10_EURSEK_fracDiff',
                'autocorr_1_EURSEK_fracDiff', 'autocorr_2_EURSEK_fracDiff',
                'autocorr_3_EURSEK_fracDiff', 'autocorr_4_EURSEK_fracDiff', 'autocorr_5_EURSEK_fracDiff','side_confidence']]

In [ ]:
sns.heatmap(X.corr())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

Sample weight the training data

In [ ]:
def mpNumCoEvents(closeIdx,t1,molecule):
    count = pd.Series(0, index=closeIdx) # just ensures we use the time indices that occur AFTER the closest time indices to t1 start and finish (t_i,0 to the very last time entry in t1)
    for tIn, tOut in t1.items():
        count.loc[tIn:tOut] += 1 # any times from the close data indices between the triple barrier times get a +1 (so we form a count for each close index time as to how many times they appear in a triple barrier interval)
    return count

def mpSampleTW(t1,numCoEvents,molecule):
    wght = pd.Series(0, index=t1.index)
    wght = wght.loc[molecule[0]:molecule[1]]
    for tIn, tOut in t1.loc[wght.index].items():
        # wght.index gives the full range of times the function will provide
        # here we go through each triple barrier interval and find the mean 1/count for each interval (where as we are checking each specific triple barrier interval, the indicator is of course always 1 for that interval)
        wght.loc[tIn] = (1./numCoEvents.loc[tIn:tOut]).mean()
    return wght

def mpSampleW(t1, numCoEvents, close, molecule):
    '''
    t1: The triple barrier times
    numCoEvents: The concurrency table indexed by raw data times (t, not i)
    close: The actual raw data for prices (using whatever bars the data is supplied in)
    molecule: The list of triple barrier (i) indices that we will use in this particular parallel processing iteration
    '''
    ret = np.log(close).diff() # log returns to ensure additivity
    wght = pd.Series(index=molecule)
    for tIn, tOut in t1.loc[wght.index].items():
        wght.loc[tIn] = (ret.loc[tIn:tOut]/numCoEvents.loc[tIn:tOut]).sum() # calculates the weight given in notes for each i
    return wght.abs() #returns weights pre normalization


def getTimeDecay(tW, clfLastW=1):
    '''
    tW: The table of uniqueness foreach barrier
    clfLastW: The user chosen c constant
    '''
    clfW = tW.sort_index().cumsum() # sorts the index for the labels and takes the cumulative sum of the uniqueness (see notes)
    # Find a and b as calculated in notes according to BCs
    if clfLastW >= 0:
        b = (1-clfLastW)/clfW.iloc[-1]
    else:
        b = 1 / ((clfLastW+1)*clfW.iloc[-1])
    a = 1 - b*clfW.iloc[-1]
    clfW = a + b*clfW # actually calculates our d function for all the values of cumulative uniqueness
    clfW[clfW < 0] = 0 # applies our max function
    print(a,b)
    return clfW

In [ ]:
events_train = events.loc[X_train.index]
concurrency = mpNumCoEvents(final_bars.index,events_train['t1'],[events_train.index[0],events_train['t1'].iloc[-1]])
avg_uniqueness = mpSampleTW(events_train['t1'],concurrency,[events_train.index[0],events_train['t1'].iloc[-1]])
weights_unnormalized=mpSampleW(events_train['t1'], concurrency, final_bars['close'],events_train['t1'].index)
td=getTimeDecay(avg_uniqueness,0.7)
weights_unnormalized_td=weights_unnormalized*td
weights_normalized=weights_unnormalized*weights_unnormalized.shape[0]/weights_unnormalized.sum()

In [ ]:
td=getTimeDecay(avg_uniqueness,0.7)
weights_unnormalized_td=weights_unnormalized*td
weights_normalized=weights_unnormalized*weights_unnormalized.shape[0]/weights_unnormalized.sum()

In [ ]:
weights_normalized

Now fit model with sample weights

In [ ]:
def get_stats(X_train, X_test, y_train, y_test, clf):
    y_positive_always = np.ones(len(X_test))
    y_train_pred = clf.predict(X_train)
    y_pred = clf.predict(X_test)
    print('Comparison')

    print('')
    print('Always trade')

    print("Confusion Matrix:")
    print(confusion_matrix(y_test,y_positive_always))

    print('Accuracy:')
    print(accuracy_score(y_test, y_positive_always))
    print('Recall:')
    print(recall_score(y_test, y_positive_always))
    print('Precision:')
    print(precision_score(y_test, y_positive_always))
    
    print('Report')
    print(classification_report(y_test,y_positive_always))

    print('')
    print('Predicted Train')

    print("Confusion Matrix:")
    print(confusion_matrix(y_train,y_train_pred))

    print('Accuracy:')
    print(accuracy_score(y_train,y_train_pred))
    print('Recall:')
    print(recall_score(y_train,y_train_pred))
    print('Precision:')
    print(precision_score(y_train,y_train_pred))
    
    print('Report')
    print(classification_report(y_train,y_train_pred))

    print('')
    print('Predicted Test')

    print("Confusion Matrix:")
    print(confusion_matrix(y_test,y_pred))

    print('Accuracy:')
    print(accuracy_score(y_test, y_pred))
    print('Recall:')
    print(recall_score(y_test, y_pred))
    print('Precision:')
    print(precision_score(y_test, y_pred))
    
    print('Report')
    print(classification_report(y_test,y_pred))

In [ ]:
# Fit without any tinkering
clf=RandomForestClassifier()
clf.fit(X_train, y_train, weights_normalized)

get_stats(X_train, X_test, y_train, y_test, clf)

Lets get a rough idea of the feature importance

In [ ]:
importances = clf.feature_importances_
importances = pd.Series(importances, index=X_train.columns)
importances.plot.bar()

We appear to be overfitting, lets do a PCA to reduce dimensionality...

#### Step 8: Feature selection, including possible PCA - as we clearly have overfitting occurring, therefore want featurers independent of labels

In [1]:
from sklearn.decomposition import PCA

# Perform PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train)

# Plot the explained variance ratio to see the importance of each component
explained_variance_ratio = pca.explained_variance_ratio_
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio)
plt.xlabel('PCA Component')
plt.ylabel('Explained Variance Ratio')
plt.title('PCA Component Importance')
plt.show()

cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Display cumulative explained variance ratio in a table
print("\nCumulative Explained Variance Ratio:")
for i, ratio in enumerate(cumulative_variance_ratio):
    print(f"Component {i + 1}: {ratio:.4f}")

# Choose a number of components based on the cumulative explained variance ratio
n_components = np.argmax(cumulative_variance_ratio >= 0.95) + 1

NameError: name 'X_train' is not defined

In [ ]:
# Re-fit PCA with the chosen number of components
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)

# Transform the test data using the same PCA
X_test_pca = pca.transform(X_test)

In [ ]:
X_test_pca

In [ ]:
y_test.value_counts()

In [ ]:
# Train a RandomForest on the PCA-transformed training data
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_pca, y_train, weights_normalized)

get_stats(X_train_pca, X_test_pca, y_train, y_test,clf)

In [ ]:
# Train a XGBoost on the PCA-transformed training data
import xgboost as xgb

xgb_cl = xgb.XGBClassifier(random_state=42)
xgb_cl.fit(X_train_pca, y_train, weights_normalized)

get_stats(X_train_pca, X_test_pca, y_train, y_test, xgb_cl)

#### Step 9: Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
param_grid = {
    "max_depth": [3, 4, 5, 7],
    "learning_rate": [0.1, 0.01, 0.05],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

xgb_cl = xgb.XGBClassifier(objective="binary:logistic")

grid_cv_xgb = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

In [ ]:
grid_cv_xgb.fit(X_train_pca, y_train)

In [ ]:
grid_cv_xgb.best_params_

#### Step 10: Fit final model with selected hyperparameters and possible PCA components, test on test data (perhaps bring in some proper backtesting principles here)

In [ ]:
xgb_cl_final = xgb.XGBClassifier(random_state=42)
xgb_cl_final.fit(X_train_pca, y_train, weights_normalized)

get_stats(X_train_pca, X_test_pca, y_train, y_test, xgb_cl_final)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

y_prob = xgb_cl_final.predict_proba(X_test_pca)[:, 1]

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# Calculate the AUC (Area Under the Curve)
auc = roc_auc_score(y_test, y_prob)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC Curve (AUC = {:.2f})'.format(auc))
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

Not sure why tghis changed slighly when re-running..

In [ ]:
y_pred = xgb_cl_final.predict(X_test_pca)
y_pred

In [ ]:
test_events = bins.loc[y_test.index]
test_events

In [ ]:
test_events['bin_pred'] = y_pred
test_events['trade_always'] = np.ones(len(test_events))
test_events['ret_trade'] = test_events['ret']*test_events['bin_pred']
test_events['ret_trade_always'] = test_events['ret']*test_events['trade_always']
test_events

In [ ]:
trades_model = test_events[test_events['ret_trade'] != 0.]
print('Model Sharpe Ratio:', trades_model['ret_trade'].mean()/trades_model['ret_trade'].std())
print('Always Trade Sharpe Ratio:', test_events['ret_trade_always'].mean()/test_events['ret_trade_always'].std())